In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
dir0 = 'file:///home/cloudera/2.kkbox_churn/data01/'

In [3]:
infile = dir0 + 'active_periods'
rdd0 = sc.textFile(infile)

In [5]:
rdd0.count()

649618

In [9]:
rdd0.take(2)

['++7jKYbuIJPXry8Oh1NcEh9fCsqcQgUaaxXsgG15kMg=,2016-03-12_2016-03-22',
 '+0RJtbyhoPAHPa+34MkYcE2Ox0cjMgJOTXMXVBYgkJE=,2016-01-13_2017-02-27']

In [4]:
rdd1 = rdd0.map(lambda line: line.split(','))
rdd1.take(5)

[['++7jKYbuIJPXry8Oh1NcEh9fCsqcQgUaaxXsgG15kMg=', '2016-03-12_2016-03-22'],
 ['+0RJtbyhoPAHPa+34MkYcE2Ox0cjMgJOTXMXVBYgkJE=', '2016-01-13_2017-02-27'],
 ['+3tRPOto/e58mdDA6oluFztHfwlqjJeNXb1vmYQjWfw=', '2016-01-18_2016-02-17'],
 ['+43UqHvqzu5STDVIuBLWryllqYYP4UrG9XyiajeGePk=',
  '2015-12-26_2017-02-28|2015-12-08_2017-01-19|2015-12-26_2017-01-29|2015-12-26_2017-02-18|2015-12-26_2017-02-08'],
 ['+4dMilwe46UsxpXXH0O7/mF3g+5fn4MAF2VTWPNzuoo=', '2016-03-05_2016-04-04']]

In [5]:
from datetime import datetime, timedelta

def date_add_one(date_str):
    d = datetime.strptime(date_str, '%Y-%m-%d') + timedelta(days = 1)
    return d.strftime('%Y-%m-%d')

def date_minus_one(date_str):
    d = datetime.strptime(date_str, '%Y-%m-%d') - timedelta(days = 1)
    return d.strftime('%Y-%m-%d')

def func(periods_str):
    '''
    Return [init, final, gaps]
    '''
    periods = periods_str.split('|')
    
    if len(periods) == 1:
        return periods[0].split('_') + [[]]
    
    ###
    shifts = {}  # e.g., {'2016-01-23': 1, '2016-01-25': -1}
                 #        ^^^^^^^^^^^^00:00
    for p in periods:
        tmp = p.split('_')
        start = tmp[0]
        end = date_add_one(tmp[1])
        
        #
        if start in shifts:
            shifts[start] += 1
        else:
            shifts[start] = 1
    
        if end in shifts:
            shifts[end] -= 1
        else:
            shifts[end] = -1
        
    shifts_1 = sorted(shifts.items())  # a list of tuples, like [('2015-12-08', 1), ('2015-12-26', 4), ...]
    if len(shifts_1) < 2:
        return ['', '', []]
    
    blocks = []
    state = 0
    for sh in shifts_1:
        if sh[1] == 0:
            continue
        state_prev = state
        state += sh[1]
        if state_prev == 0 and state > 0:
            block_start = sh[0]
        elif state_prev > 0 and state == 0:
            block_end = sh[0]
            blocks.append((block_start, block_end))
    # now blocks is like [('2015-12-08', '2016-03-01'), ('2016-3-18', '2016-4-16')]
    
    #
    init = blocks[0][0]
    final = date_minus_one(blocks[-1][1])
    
    gaps = []
    for i in range(len(blocks)-1):
        gaps.append((blocks[i][1], date_minus_one(blocks[i+1][0])))
    
    return [init, final, gaps]

In [6]:
rdd2 = rdd1.map(lambda rec: [rec[0]] + func(rec[1]))
rdd2.take(5)

[['++7jKYbuIJPXry8Oh1NcEh9fCsqcQgUaaxXsgG15kMg=',
  '2016-03-12',
  '2016-03-22',
  []],
 ['+0RJtbyhoPAHPa+34MkYcE2Ox0cjMgJOTXMXVBYgkJE=',
  '2016-01-13',
  '2017-02-27',
  []],
 ['+3tRPOto/e58mdDA6oluFztHfwlqjJeNXb1vmYQjWfw=',
  '2016-01-18',
  '2016-02-17',
  []],
 ['+43UqHvqzu5STDVIuBLWryllqYYP4UrG9XyiajeGePk=',
  '2015-12-08',
  '2017-02-28',
  []],
 ['+4dMilwe46UsxpXXH0O7/mF3g+5fn4MAF2VTWPNzuoo=',
  '2016-03-05',
  '2016-04-04',
  []]]

In [110]:
rdd3 = rdd2.map(lambda rec: rec[0:3] + [len(rec[3])])
df4 = rdd3.toDF()
df4.show(5)

+--------------------+----------+----------+---+
|                  _1|        _2|        _3| _4|
+--------------------+----------+----------+---+
|++7jKYbuIJPXry8Oh...|2016-03-12|2016-03-22|  0|
|+0RJtbyhoPAHPa+34...|2016-01-13|2017-02-27|  0|
|+3tRPOto/e58mdDA6...|2016-01-18|2016-02-17|  0|
|+43UqHvqzu5STDVIu...|2015-12-08|2017-02-28|  0|
|+4dMilwe46UsxpXXH...|2016-03-05|2016-04-04|  0|
+--------------------+----------+----------+---+
only showing top 5 rows



In [80]:
from pyspark.sql.functions import max
df4.select(max('_4')).show()

# max number of gaps:

+-------+
|max(_4)|
+-------+
|     24|
+-------+



In [71]:
df4.where('_4 > 0').show(10, truncate=False)

+--------------------------------------------+----------+----------+---+
|_1                                          |_2        |_3        |_4 |
+--------------------------------------------+----------+----------+---+
|+BauTESekzh7dOwBqraMw7OFo8WDD5nSCx5lRV3+f20=|2016-02-22|2016-04-26|1  |
|+MjSIVqd6qvlkKaP36fgvZYpm7Ohuxdu6Adpu/1AltE=|2015-12-08|2016-02-16|1  |
|+R423gwTlAs/eYBATR7PO6ZNh7moqhzA8xdDycFoBzA=|2016-11-08|2017-03-24|2  |
|+Ub6MOxsZzLGHQY7PboV2+WynMkYNw4EMb/6OBm1hAI=|2016-01-05|2017-03-18|1  |
|+Vzca1ck/BX03oBYaZ6itKg0TRbjlTguSvBSoklmQdM=|2015-11-17|2017-03-08|9  |
|+oeGl0T3RuNNupdXwSr1tGfxCK+GXm+iBzm7Ro51VCw=|2016-11-01|2017-02-26|3  |
|+s105VHAoSwRpH4V57fBUpnjVYsTyKKVDyAYSq60T7M=|2016-03-28|2016-12-19|1  |
|+tFaz7anKlaqoHvlSemlCq0Yyfl7BNmURQ2Cyr41L8U=|2015-11-22|2017-02-26|3  |
|+zYFX6MrmMLTsTNsnBPpBGK8ALau35WeAjOhn65Meyc=|2016-02-22|2016-08-05|2  |
|/2MZyij2F9o/eA1bCIIZU+MROmm7qf9MjjaXL/9F2zo=|2015-11-18|2017-03-03|4  |
+--------------------------------------------+-----

In [72]:
rdd1.filter(lambda rec: rec[0]=='+R423gwTlAs/eYBATR7PO6ZNh7moqhzA8xdDycFoBzA=').collect()

[['+R423gwTlAs/eYBATR7PO6ZNh7moqhzA8xdDycFoBzA=',
  '2017-02-21_2017-03-23|2016-11-22_2016-12-22|2016-11-08_2016-11-15|2016-12-23_2017-01-22']]

In [82]:
rdd2.filter(lambda rec: rec[0]=='+R423gwTlAs/eYBATR7PO6ZNh7moqhzA8xdDycFoBzA=').collect()

[['+R423gwTlAs/eYBATR7PO6ZNh7moqhzA8xdDycFoBzA=',
  '2016-11-08',
  '2017-03-24',
  [('2016-11-16', '2016-11-22'), ('2017-01-23', '2017-02-21')]]]

# Output

In [7]:
def gaps_to_string(gaps):
    gaps_string_list = ['_'.join(gap) for gap in gaps]
    return '|'.join(gaps_string_list)

#gaps_to_string([('2016-11-16', '2016-11-22'), ('2017-01-23', '2017-02-21')])

rdd3 = rdd2.map(lambda rec: rec[0:3] + [gaps_to_string(rec[3])])
rdd3.take(5)

[['++7jKYbuIJPXry8Oh1NcEh9fCsqcQgUaaxXsgG15kMg=',
  '2016-03-12',
  '2016-03-22',
  ''],
 ['+0RJtbyhoPAHPa+34MkYcE2Ox0cjMgJOTXMXVBYgkJE=',
  '2016-01-13',
  '2017-02-27',
  ''],
 ['+3tRPOto/e58mdDA6oluFztHfwlqjJeNXb1vmYQjWfw=',
  '2016-01-18',
  '2016-02-17',
  ''],
 ['+43UqHvqzu5STDVIuBLWryllqYYP4UrG9XyiajeGePk=',
  '2015-12-08',
  '2017-02-28',
  ''],
 ['+4dMilwe46UsxpXXH0O7/mF3g+5fn4MAF2VTWPNzuoo=',
  '2016-03-05',
  '2016-04-04',
  '']]

In [13]:
from pyspark.sql import Row
rdd4 = rdd3.map(lambda rec: Row(msno=rec[0], 
                                first_start_date=rec[1],
                                last_expire_date=rec[2],
                                gaps=rec[3]
                               ))
df5 = rdd4.toDF().select('msno', 'first_start_date', 'last_expire_date', 'gaps')
df5.show(35)

+--------------------+----------------+----------------+--------------------+
|                msno|first_start_date|last_expire_date|                gaps|
+--------------------+----------------+----------------+--------------------+
|++7jKYbuIJPXry8Oh...|      2016-03-12|      2016-03-22|                    |
|+0RJtbyhoPAHPa+34...|      2016-01-13|      2017-02-27|                    |
|+3tRPOto/e58mdDA6...|      2016-01-18|      2016-02-17|                    |
|+43UqHvqzu5STDVIu...|      2015-12-08|      2017-02-28|                    |
|+4dMilwe46UsxpXXH...|      2016-03-05|      2016-04-04|                    |
|+5p5fm9oHH8ukkNG1...|      2016-06-08|      2016-07-08|                    |
|+7LsJK3q0POdtDWvw...|      2016-06-03|      2016-07-01|                    |
|+7f07tkg6Q6sPq1MX...|      2016-11-17|      2016-11-24|                    |
|+8suRKxAwKNflQAfd...|      2016-11-05|      2016-11-12|                    |
|+98dadrcy1re8eJQo...|      2016-10-24|      2016-10-31|        

In [14]:
outfile = dir0 + 'member_active_gaps'
df5.write.format('csv').option('header','true').save(outfile)